In [1]:
from solve_distrib_rad import *
from IPython.core.magic import register_cell_magic
import time

@register_cell_magic
def skip(line, cell):
    return


Test to do (convergence tests to accompany with execution time measure)
1. $\gamma$ binning
2. t binning
3. switch to monoE regime
4. cooling regimes

#### 1. $\gamma$ binning
- look at peak $\nu F_\nu$ vs value at high Ng
- choose Ng by convergence (comparing vs a high Ng)
- again with trapezoidal rule
- compare efficiencies

In [2]:
key, k = 'test', 450
j = 0
Ng = 20
data, env = get_cellData_withDistrib_analytic(key, k, Ng=Ng)
nub = np.logspace(-3, 2, 100)
nuobs = env.nu0 * nub
Tobs = env.Ts
cell0, cell, cell_next = data.iloc[0], data.iloc[j], data.iloc[j+1]
K0 = norm_plaw_distrib(cell0.gmin, cell0.gmax, env.psyn)
nu_B0 = get_variable(cell0, 'nu_B', env)
tnu_arr = nuobs/nu_B0
steps = split_hydrostep(cell, cell_next, env)
step = steps.iloc[0]

rescaling by factor 1.0e+00


In [3]:
%%skip
gma_keys = [key for key in steps.keys() if 'gm' in key]
fig, ax = plt.subplots()
for key in gma_keys:
  ax.scatter(steps.tt, steps[key])
ax.set_xscale('log')
ax.set_yscale('log')

In [4]:
%%skip
p   = env.psyn
gmin, gmax = step.gmin, step.gmax
gma_keys = [key for key in step.keys() if 'gm' in key]
gmas_arr = step[gma_keys].to_numpy(copy=True)
Pmax = get_variable(step, "Pmax", env)
K = K0
P1 = Pnu_instant(tnu_arr, gmin, gmax, env)
P2 = Pnu_instant_bins(tnu_arr, gmas_arr, env)

fig, ax = plt.subplots()
ax.loglog(tnu_arr, P1, label='quad')
ax.loglog(tnu_arr, P2, label='bins')
ax.legend()

In [4]:
def test_Fnu_Ng(Ng):
  key, k = 'test', 450
  data, env = get_cellData_withDistrib_analytic(key, k, Ng=Ng)
  nuobs = env.nu0 * nub
  Tobs = env.Ts
  cell0, cell, cell_next = data.iloc[0], data.iloc[j], data.iloc[j+1]
  K0 = norm_plaw_distrib(cell0.gmin, cell0.gmax, env.psyn)
  F = get_Fnu_hydrostep(nuobs, Tobs, cell, cell_next, K0, env)
  return nub*F


In [6]:
start = time.perf_counter()
nF_ref = test_Fnu_Ng(0)
exec_ref = time.perf_counter() - start
pk_ref = nF_ref.max()

rescaling by factor 1.0e+00


In [ ]:
%%skip
nF_20 = test_Fnu_Ng(20)
err = (nF_20-nF_ref)/nF_ref
print(err.max())

In [8]:
%%skip
nF_100 = test_Fnu_Ng(100)
err = (nF_100-nF_ref)/nF_ref
print(err.max())

In [9]:
%%skip
fig, ax = plt.subplots()
ax.loglog(nub, nF_ref, label='quad')
ax.loglog(nub, nF_20, label='bins')
ax.set_ylabel('$\\nu F_\\nu/\\nu_0F_0$')
ax.set_ylabel('$\\nu/\\nu_0$')
ax.legend()

err = (nF_20-nF_ref)/nF_ref
ax2 = ax.twinx()
ax2.loglog(nub, err, c='k', lw=.9)
ax2.set_ylabel('err')

fig.tight_layout()

In [7]:

errs, times = [], []
Ngs = [5, 10, 20, 50, 100]
for Ng in Ngs:
  start = time.perf_counter()
  nF = test_Fnu_Ng(Ng)
  elapsed = time.perf_counter() - start
  times.append(elapsed)
  pk = nF.max()
  err = (pk-pk_ref)/pk_ref
  errs.append(err)

rescaling by factor 1.0e+00
rescaling by factor 1.0e+00
rescaling by factor 1.0e+00
rescaling by factor 1.0e+00
rescaling by factor 1.0e+00


In [8]:

fig, ax = plt.subplots()
ax.scatter(Ngs, errs)
ax.set_ylabel('err')
ax.set_xlabel('$N_\\gamma$')
ax.set_yscale('log')
ax.tick_params(axis='y', colors='C0')
ax.yaxis.label.set_color('C0')
ax2 = ax.twinx()
ax2.set_ylabel('$t_{\\rm exec}$ (s)')
ax2.axhline(exec_ref, ls=':', c='k', label='time (quad)')
ax2.scatter(Ngs, times, marker='x', c='k')
ax2.legend(loc='upper center')
fig.tight_layout()

#### 2. $t$ binning
- look at peak $\nu F_\nu$ vs value at high Nt
- choose right r_ref, by convergence (again)
- try with different cooling regimes

In [12]:
r_refs = np.geomspace(1.2, 1, num=20, endpoint=False)

Number of time bins vs $\gamma_{\max}$ ratio

In [13]:
%%skip
Nt = []
for r_ref in r_refs:
  steps = split_hydrostep(cell, cell_next, env, r_ref, Nmax=1000)
  Nt.append(len(steps))

fig, ax = plt.subplots()
ax.loglog(r_refs, Nt)
ax.set_xlabel('$r_{\\rm ref}$')
ax.set_ylabel('$N_t$', rotation='horizontal')

Relative difference at peak $\nu F_\nu$ vs $\gamma_{\max}$ ratio (NB: calc time of several minutes)

In [ ]:
%%skip
start = time.perf_counter()
F_ref = get_Fnu_hydrostep(nuobs, Tobs, cell, cell_next, K0, env, r_ref=r_refs[-1], Nmax=1000)
exec_ref = time.perf_counter() - start
pk_ref = (nub*F_ref).max()

In [ ]:
%%skip
errs, times = [], []
for r in r_refs:
  start = time.perf_counter()
  F = get_Fnu_hydrostep(nuobs, Tobs, cell, cell_next, K0, env, r_ref=r, Nmax=1000)
  elapsed = time.perf_counter() - start
  times.append(elapsed)
  F *= nub
  err = (F.max()-pk_ref)/pk_ref
  errs.append(err)


In [ ]:
%%skip
fig, ax = plt.subplots()
ax.scatter(r_refs, errs)
ax.set_ylabel('err')
ax.set_xlabel('$r_{\\rm ref}$')
ax.set_yscale('log')
ax.tick_params(axis='y', colors='C0')
ax.yaxis.label.set_color('C0')
ax2 = ax.twinx()
ax2.set_ylabel('$t_{\\rm exec}$ (s)')
ax2.scatter(r_refs, times, marker='x', c='k')
#ax2.axhline(exec_ref, ls=':', c='k', label='time (quad)')
#ax2.legend(loc='upper center')
fig.tight_layout()

In [17]:
%%skip
fig, ax = plt.subplots()
ax.semilogy(r_refs, Nt*err)
ax2 = ax.twinx()
ax2.semilogy(r_refs, Nt)
ax2.set_ylabel("$N_t$")
ax.set_xlabel('$r_{\\rm ref}$')
ax.set_title('$N_t \\times err$')

In [18]:
# check gma_min/max (with t binning) over cell history
%%skip
scalefac = 0
Ng = 0
data, env = get_cellData_withDistrib_analytic(key, k, Ng, scalefac)
r_ref = 1.05
fig, ax = plt.subplots()
trans = transx(ax)
ax.set_xlabel('$\\tilde{t}$', rotation='horizontal')
ax.set_ylabel('$\\gamma$')
Nj = len(data)
tt = data.tt.to_numpy(copy=True)
gmin0, gmax0 = data.iloc[0][['gmin', 'gmax']]
gmins = data.gmin.to_numpy()
gmins_th = gamma_synCooled(tt, gmin0)
gmaxs = data.gmax.to_numpy()
gmaxs_th = gamma_synCooled(tt, gmax0)
ax.axhline(gmin0, c='C1', ls=':', xmax=0.1)
ax.text(-0.01, gmin0, '$\\gamma_{\\min,0}$', c='C1', ha='right', transform=trans)
ax.axhline(gmax0, c='C3', ls=':', xmax=0.1)
ax.text(-0.01, gmax0, '$\\gamma_{\\max,0}$', c='C3', ha='right', transform=trans)
ax.loglog(tt, gmins, c='C1')
#ax.loglog(tt, gmins_th, c='k', ls='--', lw=.9)
ax.loglog(tt, gmaxs, c='C3')
#ax.loglog(tt, gmaxs_th, c='k', ls='--', lw=.9)
for j in range(Nj-1):
  cell, cell_next = data.iloc[j], data.iloc[j+1]
  steps = split_hydrostep(cell, cell_next, env, r_ref, Nmax=1000)
  ax.scatter(steps.tt, steps.gmax, c='C3', marker='x')
  ax.scatter(steps.tt, steps.gmin, c='C1', marker='x')
  ax.scatter(cell.tt, cell.gmax, c='k', marker='x')
  ax.scatter(cell.tt, cell.gmin, c='k', marker='x')

ax.plot([], [], c='C1', label='$\\gamma_\\min$')
ax.plot([], [], c='C3', label='$\\gamma_\\max$')
#ax.plot([], [], c='k', ls='--', label='cooling function')
ax.scatter([], [], marker='x', c='k', label='bins')
ax.legend()


UsageError: Line magic function `%%skip` not found.


### switch to monoE regime
- look at max relative error vs fully integral $\nu F_\nu$
- choose the correct width_tol

In [ ]:
%%skip
start = time.perf_counter()
F_ref = get_Fnu_hydrostep(nuobs, Tobs, cell, cell_next, K0, env, width_tol=0.9, r_ref=1.2, Nmax=100)
exec_ref = time.perf_counter() - start
pk_ref = (nub*F_ref).max()

In [ ]:
%%skip
tols = np.geomspace(2, 1, num=30)
errs, times = [], []
for w in tols:
  start = time.perf_counter()
  F =  get_Fnu_hydrostep(nuobs, Tobs, cell, cell_next, K0, env, width_tol=w, r_ref=1.2, Nmax=100)
  elapsed = time.perf_counter() - start
  times.append(elapsed)
  F *= nub
  err = np.abs(F.max()-pk_ref)/pk_ref
  errs.append(err)


ValueError: operands could not be broadcast together with shapes (100,) (200,) (100,) 

In [ ]:
%%skip
fig, ax = plt.subplots()
ax.scatter(tols, errs)
ax.set_ylabel('err')
ax.set_xlabel('$(\\gamma_{\\max}/\gamma_{\\min})_{\\rm ref}$')
ax.set_yscale('log')
ax.tick_params(axis='y', colors='C0')
ax.yaxis.label.set_color('C0')
ax2 = ax.twinx()
ax2.set_ylabel('$t_{\\rm exec}$ (s)')
ax2.scatter(tols, times, marker='x', c='k')
fig.tight_layout()

ValueError: x and y must be the same size

### 4. cooling regimes
The bins have indeed solved everything, including exploding calculation time.

In [9]:
key, k = 'test', 450
scales = [0, 1, 2, 3, 4]
nub = np.logspace(-4, 8, 200)

In [47]:
def test_Fnu_scalefac(scalefac):
  key, k = 'test', 450
  data, env = get_cellData_withDistrib_analytic(key, k, scalefac=scalefac)
  nuobs = env.nu0 * nub
  Tobs = env.Ts
  cell0, cell, cell_next = data.iloc[0], data.iloc[j], data.iloc[j+1]
  K0 = norm_plaw_distrib(cell0.gmin, cell0.gmax, env.psyn)
  start = time.perf_counter()
  F = get_Fnu_hydrostep(nuobs, Tobs, cell, cell_next, K0, env)
  elapsed = time.perf_counter() - start
  nup_B = get_variable(cell, 'nup_B', env)
  nup_m = get_variable(cell, 'nup_m', env)
  x_B = nup_B/nup_m
  x_M = x_B*cell.gmax**2
  x_c = x_B*cell_next.gmax**2
  y = spectrum_bplaw(nub, x_B, x_c, 1, x_M, env.psyn)
  i0 = np.searchsorted(nub, 1)
  y *= F[i0]/y[i0]
  return nub*F, elapsed, nub * y

In [10]:
def test_Fnu_scalefac_cell(scalefac):
  key, k = 'test', 450
  data, env = get_cellData_withDistrib_analytic(key, k, scalefac=scalefac)
  nuobs = env.nu0 * nub
  Tobs = env.Ts
  cell0, cellf = data.iloc[0], data.iloc[-1]
  start = time.perf_counter()
  F = get_Fnu_cell(nuobs, Tobs, data, env)
  elapsed = time.perf_counter() - start
  nup_B = get_variable(cell0, 'nup_B', env)
  nup_m = get_variable(cell0, 'nup_m', env)
  x_B = nup_B/nup_m
  x_M = x_B*cell0.gmax**2
  x_c = x_B*cellf.gmax**2
  y = spectrum_bplaw(nub, x_B, x_c, 1, x_M, env.psyn)
  i0 = np.searchsorted(nub, 1)
  y *= F[i0]/y[i0]
  return nub*F, elapsed, nub * y

In [11]:
times, fluxes, bpl = [], [], []
for sc in scales:
  nF, t, y = test_Fnu_scalefac_cell(sc)
  times.append(t)
  fluxes.append(nF)
  bpl.append(y)
print(times)

rescaling by factor 1.0e+00
Calculating over 22 sim iterations
VFC
rescaling by factor 1.0e+01
Calculating over 224 sim iterations
FC
rescaling by factor 1.0e+02
Calculating over 224 sim iterations
SC
rescaling by factor 1.0e+03
Calculating over 224 sim iterations
SC
rescaling by factor 1.0e+04
Calculating over 224 sim iterations
SC
[1.0824858289997792, 8.920078625000315, 27.77510597699984, 24.69179868700121, 24.268306920001123]


In [12]:
fig, ax = plt.subplots()
colors = [f'C{i}' for i in range(len(fluxes))]
for nF, sc, y, col in zip(fluxes, scales, bpl, colors):
  ax.loglog(nub, nF, c=col, label=f'{sc}')
  ax.loglog(nub, y, c=col, ls='--')
ax.set_xlabel("$\\nu/\\nu_0$")
ax.set_ylabel("$\\nu F_\\nu/\\nu_0F_0$")
ax.legend(title='$\\log_{10}(R_0/R_{0,ref})$')

In [ ]:
%%skip
fig, ax = plt.subplots()
env0 = MyEnv(key)
var = 'nu0'
arr, names = [], []
for scalefac in scales:
  env = MyEnv(key, scalefac)
  val = getattr(env, var)
  print(val)
  arr.append(val)
ax.semilogy(scales, arr)
ax.set_yscale('log')

1.0847466381459125e+20
1.0847466381457146e+17
108474663814591.27
108474663814.59125


In [ ]:
%%skip
fig, ax = plt.subplots()
var = 'nu_B'
norm = ''
arr, names = [], []
for scalefac in scales:
  data, env = get_cellData_withDistrib_analytic(key, k, scalefac)
  cell0 = data.iloc[0]
  renorm = getattr(env, norm) if norm else 1.
  val = get_variable(cell0, var, env)/renorm
  print(val)
  arr.append(val)
ax.semilogy(scales, arr)
ax.set_yscale('log')

In [ ]:
%%skip
fig, ax = plt.subplots()
ax.set_xlabel('$\\nu/\\nu_0$')
ax.set_ylabel('$\\nu F_\\nu/\\nu_0F_0$')
fig.tight_layout()

def func_(scalefac):
  data, env = get_cellData_withDistrib_analytic(key, k, scalefac)
  Tobs = env.Ts
  nuobs = nub*env.nu0
  Fnu = get_Fnu_cell_v2(nuobs, Tobs, data, env)
  return Fnu

In [ ]:
%%skip
ax.legend(title='$\\log10(R_0/R_{\\rm 0,vFC})$')
plt.savefig('../figures/coolregimes_test.png')